In [ ]:
import os
import torch
from PIL import Image
from torchvision import transforms
import gdown
import zipfile

In [ ]:
file_id = "file_id"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "texture_cvl_before.zip", quiet=False)
with zipfile.ZipFile("texture_cvl_before.zip", 'r') as zip_ref:
    zip_ref.extractall("texture_cvl_before")

In [ ]:
CVL_SRC = "texture_cvl_before"
PRE_CVL = "texture_cvl"
IMG_SIZE = (384, 384)

transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5),
                         std =(0.5, 0.5, 0.5)),
])

def process_and_save(src_root, dst_root):

    for root, _, files in os.walk(src_root):
        for fname in files:
            if not fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue

            rel_dir = os.path.relpath(root, src_root)
            dst_dir = os.path.join(dst_root, rel_dir)
            os.makedirs(dst_dir, exist_ok=True)

            base, _ = os.path.splitext(fname)
            src_path = os.path.join(root, fname)
            dst_path = os.path.join(dst_dir, base + ".pt")

            try:
                img = Image.open(src_path)
                tensor = transform(img)
                torch.save(tensor, dst_path)
                print(f"Saved: {dst_path}")
            except Exception as e:
                print(f"Error processing {src_path}: {e}")

process_and_save(CVL_SRC, PRE_CVL)

In [ ]:
!zip -r texture_cvl.zip texture_cvl

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

target_directory = "/content/drive/MyDrive/PATH"
!cp texture_cvl.zip "{target_directory}/"